# NOTE:  THIS NOTEBOOK RUNS FOR 30+ MINUTES.  

# PLEASE BE PATIENT

# Train a Model with SageMaker Autopilot

We will use Autopilot to predict the gender of the record. Autopilot implements a transparent approach to AutoML. 

For more details on Autopilot, have a look at this [**Amazon Science Publication**](https://www.amazon.science/publications/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale).

[![](img/amazon_science.png)](https://www.amazon.science/publications/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale)


<img src="img/autopilot-transparent.png" width="80%" align="left">

# Introduction

Amazon SageMaker Autopilot is a service to perform automated machine learning (AutoML) on your datasets.  Autopilot is available through the UI or AWS SDK.  In this notebook, we will use the AWS SDK to create and deploy a text processing and star rating classification machine learning pipeline.

# Setup

Let's start by specifying:

* The S3 bucket and prefix to use to train our model.  _Note:  This should be in the same region as this notebook._
* The IAM role of this notebook needs access to your data.

# Notes
* This notebook will take some time to finish. 

* You can start this notebook and continue to the next notebooks whenever you are waiting for the current notebook to finish.

# Checking Pre-Requisites From Previous Notebook

In [1]:
%store -r autopilot_train_s3_uri

In [2]:
try:
    autopilot_train_s3_uri
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

[OK]


In [3]:
print(autopilot_train_s3_uri)

s3://ada-dev/xiaotian/ml/autopilot/data/train_data.csv


In [ ]:
if not autopilot_train_s3_uri:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

In [4]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session(default_bucket = "ada-dev")
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [5]:
bucket = sess.default_bucket()

In [6]:
bucket

'ada-dev'

# Training Data

In [56]:
!aws s3 ls $autopilot_train_s3_uri

2020-11-13 09:17:30     661027 train_data.csv


## See our prepared training data which we use as input for Autopilot

In [55]:
!aws s3 cp $autopilot_train_s3_uri ./tmp/

download: s3://ada-dev/xiaotian/ml/autopilot/data/train_data.csv to tmp/train_data.csv


In [7]:
import csv

df = pd.read_csv('./tmp/train_data.csv')
df.head()

,ifa,bundle_vec,persona_segment_vec,persona_L1_vec,persona_L2_vec,persona_L3_vec,device_vendor_vec,device_name_vec,device_manufacturer_vec,device_model_vec,device_year_of_release_vec,dev_platform_vec,major_os_vec,label_int
0,0618c523-1889-4b20-82fc-246c78718689,"{'type': 0, 'size': 7900, 'indices': array([ ...","{'type': 0, 'size': 57, 'indices': array([ 0, ...","{'type': 0, 'size': 19, 'indices': array([ 7, ...","{'type': 0, 'size': 85, 'indices': array([ 8, ...","{'type': 0, 'size': 60, 'indices': array([8], ...","{'type': 0, 'size': 21, 'indices': array([1], ...","{'type': 0, 'size': 249, 'indices': array([0],...","{'type': 0, 'size': 21, 'indices': array([1], ...","{'type': 0, 'size': 475, 'indices': array([89]...","{'type': 0, 'size': 7, 'indices': array([3], d...","{'type': 0, 'size': 2, 'indices': array([0], d...","{'type': 0, 'size': 7, 'indices': array([2], d...",3
1,15af96ba-3fe0-4833-a280-c19194a29d80,"{'type': 0, 'size': 7900, 'indices': array([ ...","{'type': 0, 'size': 57, 'indices': array([0, 3...","{'type': 0, 'size': 19, 'indices': array([0], ...","{'type': 0, 'size': 85, 'indices': array([1], ...","{'type': 0, 'size': 60, 'indices': array([], d...","{'type': 0, 'size': 21, 'indices': array([0], ...","{'type': 0, 'size': 249, 'indices': array([2],...","{'type': 0, 'size': 21, 'indices': array([0], ...","{'type': 0, 'size': 475, 'indices': array([1],...","{'type': 0, 'size': 7, 'indices': array([0], d...","{'type': 0, 'size': 2, 'indices': array([0], d...","{'type': 0, 'size': 7, 'indices': array([0], d...",3
2,2c4ede0c-32ef-4c69-964c-545bacc2fdef,"{'type': 0, 'size': 7900, 'indices': array([ ...","{'type': 0, 'size': 57, 'indices': array([ 0, ...","{'type': 0, 'size': 19, 'indices': array([ 1, ...","{'type': 0, 'size': 85, 'indices': array([ 2, ...","{'type': 0, 'size': 60, 'indices': array([0], ...","{'type': 0, 'size': 21, 'indices': array([4], ...","{'type': 0, 'size': 249, 'indices': array([49]...","{'type': 0, 'size': 21, 'indices': array([4], ...","{'type': 0, 'size': 475, 'indices': array([67]...","{'type': 0, 'size': 7, 'indices': array([1], d...","{'type': 0, 'size': 2, 'indices': array([0], d...","{'type': 0, 'size': 7, 'indices': array([0], d...",1
3,80bdfeba-5cb0-414a-87a6-aac24451a20a,"{'type': 0, 'size': 7900, 'indices': array([ ...","{'type': 0, 'size': 57, 'indices': array([ 0, ...","{'type': 0, 'size': 19, 'indices': array([ 0, ...","{'type': 0, 'size': 85, 'indices': array([ 3, ...","{'type': 0, 'size': 60, 'indices': array([4], ...","{'type': 0, 'size': 21, 'indices': array([0], ...","{'type': 0, 'size': 249, 'indices': array([3],...","{'type': 0, 'size': 21, 'indices': array([0], ...","{'type': 0, 'size': 475, 'indices': array([2],...","{'type': 0, 'size': 7, 'indices': array([0], d...","{'type': 0, 'size': 2, 'indices': array([0], d...","{'type': 0, 'size': 7, 'indices': array([0], d...",3
4,ad0a78ef-f66b-4da4-a2b2-495b07d1ebf3,"{'type': 0, 'size': 7900, 'indices': array([ ...","{'type': 0, 'size': 57, 'indices': array([ 0, ...","{'type': 0, 'size': 19, 'indices': array([ 3, ...","{'type': 0, 'size': 85, 'indices': array([ 3, ...","{'type': 0, 'size': 60, 'indices': array([], d...","{'type': 0, 'size': 21, 'indices': array([1], ...","{'type': 0, 'size': 249, 'indices': array([0],...","{'type': 0, 'size': 21, 'indices': array([1], ...","{'type': 0, 'size': 475, 'indices': array([27]...","{'type': 0, 'size': 7, 'indices': array([2], d...","{'type': 0, 'size': 2, 'indices': array([0], d...","{'type': 0, 'size': 7, 'indices': array([0], d...",1


# Setup the S3 Location for the Autopilot-Generated Assets 
This include Jupyter Notebooks (Analysis), Python Scripts (Feature Engineering), and Trained Models.

In [8]:
prefix_model_output = 'xiaotian/ml/autopilot/model'

model_output_s3_uri = 's3://{}/{}'.format(bucket, prefix_model_output)

print(model_output_s3_uri)

s3://ada-dev/xiaotian/ml/autopilot/model


In [9]:
max_candidates = 3 #select only 3 top candidates

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 900,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 5400
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(autopilot_train_s3_uri)
        }
      },
      'TargetAttributeName': 'label_int' #specify the target column
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(model_output_s3_uri)
}

# Check For Existing Autopilot Jobs

In [10]:
existing_jobs_response = sm.list_auto_ml_jobs()

In [11]:
existing_jobs_response

{'AutoMLJobSummaries': [{'AutoMLJobName': 'automl-dm-28-08-28-05',
   'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:331041401449:automl-job/automl-dm-28-08-28-05',
   'AutoMLJobStatus': 'Completed',
   'AutoMLJobSecondaryStatus': 'MaxCandidatesReached',
   'CreationTime': datetime.datetime(2020, 8, 28, 8, 28, 8, 738000, tzinfo=tzlocal()),
   'EndTime': datetime.datetime(2020, 8, 28, 8, 50, 10, 734000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 8, 28, 8, 50, 10, 769000, tzinfo=tzlocal())},
  {'AutoMLJobName': 'automl-dm-12-02-54-20',
   'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:331041401449:automl-job/automl-dm-12-02-54-20',
   'AutoMLJobStatus': 'Completed',
   'AutoMLJobSecondaryStatus': 'MaxCandidatesReached',
   'CreationTime': datetime.datetime(2020, 11, 12, 2, 54, 45, 128000, tzinfo=tzlocal()),
   'EndTime': datetime.datetime(2020, 11, 12, 3, 18, 8, 575000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 11, 12, 3, 18, 8, 606000, tzinfo=t

In [12]:
num_existing_jobs = 0 
running_jobs = 0

if 'AutoMLJobSummaries' in existing_jobs_response.keys():
    job_list = existing_jobs_response['AutoMLJobSummaries']
    num_existing_jobs = len(job_list)
    # print('[INFO] You already created {} Autopilot job(s) in this account.'.format(num_existing_jobs))
    for j in job_list:
        if 'AutoMLJobStatus' in j.keys():                
            if j['AutoMLJobStatus'] == 'InProgress':
                running_jobs = running_jobs + 1
    print('[INFO] You have {} Autopilot job(s) currently running << Should be 0 jobs.'.format(running_jobs))
else:
    print('[OK] Please continue.')

[INFO] You have 0 Autopilot job(s) currently running << Should be 0 jobs.


# Launch the SageMaker Autopilot Job

In [13]:
from time import gmtime, strftime, sleep

In [14]:
# %store -r auto_ml_job_name
try:
    auto_ml_job_name
except NameError:    
    timestamp_suffix = strftime('%-m-%d-%H-%M', gmtime())
    auto_ml_job_name = 'automl-dm-' + timestamp_suffix
    print('Created AutoMLJobName: ' + auto_ml_job_name)

Created AutoMLJobName: automl-dm-11-14-13-48


In [15]:
print(auto_ml_job_name)

automl-dm-11-14-13-48


In [16]:
%store auto_ml_job_name

Stored 'auto_ml_job_name' (str)


In [17]:
max_running_jobs = 1

if running_jobs < max_running_jobs: # Limiting to max. 1 Jobs
    try:
        sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                              InputDataConfig=input_data_config,
                              OutputDataConfig=output_data_config,
                              AutoMLJobConfig=job_config,
                              RoleArn=role)
        print('[OK] Autopilot Job {} created.'.format(auto_ml_job_name))
        running_jobs = running_jobs + 1
    except:
        print('[INFO] You have already launched an Autopilot job. Please continue see the output of this job.'.format(running_jobs))
else:
    print('[INFO] You have already launched {} Autopilot running job(s). Please continue see the output of the running job.'.format(running_jobs))

[OK] Autopilot Job automl-dm-11-14-13-48 created.


# Track the Progress of the Autopilot Job

SageMaker Autopilot job consists of the following high-level steps: 
* _Data Analysis_ where the data is summarized and analyzed to determine which feature engineering techniques, hyper-parameters, and models to explore.
* _Feature Engineering_ where the data is scrubbed, balanced, combined, and split into train and validation.
* _Model Training and Tuning_ where the top performing features, hyper-parameters, and models are selected and trained.

<img src="img/autopilot-steps.png" width="90%" align="left">

**Autopilot Research Paper:   
https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf**

# Analyzing Data and Generate Notebooks

In [18]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_description_response

{'AutoMLJobName': 'automl-dm-11-14-13-48',
 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:331041401449:automl-job/automl-dm-11-14-13-48',
 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
     'S3Uri': 's3://ada-dev/xiaotian/ml/autopilot/data/train_data.csv'}},
   'TargetAttributeName': 'label_int'}],
 'OutputDataConfig': {'S3OutputPath': 's3://ada-dev/xiaotian/ml/autopilot/model'},
 'RoleArn': 'arn:aws:iam::331041401449:role/ADA_SageMaker',
 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3,
   'MaxRuntimePerTrainingJobInSeconds': 900,
   'MaxAutoMLJobRuntimeInSeconds': 5400}},
 'CreationTime': datetime.datetime(2020, 11, 14, 13, 48, 31, 676000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 11, 14, 13, 48, 39, 89000, tzinfo=tzlocal()),
 'AutoMLJobStatus': 'InProgress',
 'AutoMLJobSecondaryStatus': 'AnalyzingData',
 'GenerateCandidateDefinitionsOnly': False,
 'ResponseMetadata': {'RequestId': '4e52f27e-8414-4d97-91c3-56628ffc4e

In [19]:
# job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while 'AutoMLJobStatus' not in job_description_response.keys() and 'AutoMLJobSecondaryStatus' not in job_description_response.keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print('[INFO] Autopilot Job has not yet started. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for Autopilot Job to start...')
    sleep(15)

print('[OK] AutoMLJob started.')

[OK] AutoMLJob started.


# Review the SageMaker `Processing Jobs`
* First Processing Job (Data Splitter) checks the data sanity, performs stratified shuffling and splits the data into training and validation. 
* Second Processing Job (Candidate Generator) first streams through the data to compute statistics for the dataset. Then, uses these statistics to identify the problem type, and possible types of every column-predictor: numeric, categorical, natural language, etc.

In [20]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/">Processing Jobs</a></b>'.format(region)))


# The Next Cell Will Show `InProgress` For A Few Minutes.

## _Please be patient._

In [21]:
%%time

job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('Starting', 'AnalyzingData'):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(15)
    print('[OK] Data analysis phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering
[OK] Data analysis phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:331041401449:automl-job/automl-dm-11-14-13-48",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNot

# View Generated Notebook Samples
Once data analysis is complete, SageMaker AutoPilot generates two notebooks: 
* Data Exploration
* Candidate Definition

# Waiting For Generated Notebooks

In [22]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while 'AutoMLJobArtifacts' not in job_description_response.keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print('[INFO] Autopilot Job has not yet generated the artifacts. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for AutoMLJobArtifacts...')
    sleep(15)

print('[OK] AutoMLJobArtifacts generated.')

[OK] AutoMLJobArtifacts generated.


In [23]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while 'DataExplorationNotebookLocation' not in job_description_response['AutoMLJobArtifacts'].keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print('[INFO] Autopilot Job has not yet generated the notebooks. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for DataExplorationNotebookLocation...')
    sleep(15)

print('[OK] DataExplorationNotebookLocation found.')   

[OK] DataExplorationNotebookLocation found.


In [24]:
generated_resources = job_description_response['AutoMLJobArtifacts']['DataExplorationNotebookLocation']
download_path = generated_resources.rsplit('/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')[0]
job_id = download_path.rsplit('/', 1)[-1]

In [25]:
from IPython.core.display import display, HTML

if not job_id: 
    print('No AutoMLJobArtifacts found.')
else: 
    display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/{}/sagemaker-automl-candidates/{}/">S3 Generated Resources</a></b>'.format(bucket, prefix_model_output, auto_ml_job_name, job_id)))

# Download Generated Notebooks & Code

In [26]:
print(download_path)

s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d


In [27]:
try:
    !aws s3 cp --recursive $download_path .
except:
    print('Could not download the generated resources. Make sure the path is correct.')

download: s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/generated_module/candidate_data_processors/dpp2.py to generated_module/candidate_data_processors/dpp2.py
download: s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/generated_module/candidate_data_processors/trainer.py to generated_module/candidate_data_processors/trainer.py
download: s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/generated_module/README.md to generated_module/README.md
download: s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/generated_module/candidate_data_processors/dpp1.py to generated_mod

# Review The Generated Resources:

In [81]:
!ls ./generated_module/candidate_data_processors

dpp0.py  dpp1.py  dpp2.py  sagemaker_serve.py  trainer.py


In [82]:
!ls ./notebooks

sagemaker_automl
SageMakerAutopilotCandidateDefinitionNotebook.ipynb
SageMakerAutopilotDataExplorationNotebook.ipynb


# Feature Engineering

### Watch out for SageMaker `Training Jobs` and `Batch Transform Jobs` to start. 

* This is the candidate exploration phase. 
* Each python script code for data-processing is executed inside a SageMaker framework container as a training job, followed by transform job.

Note, that feature preprocessing part of each pipeline has all hyper parameters fixed, i.e. does not require tuning, thus feature preprocessing step can be done prior runing the hyper parameter optimization job. 

It outputs up to 10 variants of transformed data, therefore algorithms for each pipeline are set to use
the respective transformed data.

<img src="img/autopilot-steps.png" width="90%" align="left">

**Autopilot Research Paper:   
https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf**

In [83]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/">Training Jobs</a></b>'.format(region)))


In [84]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/">Batch Transform Jobs</a></b>'.format(region)))


# The Next Cell Will Show `InProgress` For A Few Minutes.

## _Please be patient._ ##

In [29]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(15)
    print('[OK] Feature engineering phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

Completed
MaxCandidatesReached
[OK] Feature engineering phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:331041401449:automl-job/automl-dm-11-14-13-48",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 5400,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 900
        }
    },
    "AutoMLJobName": "automl-dm-11-14-13-48",
    "AutoMLJ

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

# Model Training and Tuning

### Watch out for a SageMaker`Hyperparameter Tuning Job` and various `Training Jobs` to start. 

* All algorithms are optimized using a SageMaker Hyperparameter Tuning job. 
* Up to 250 training jobs (based on number of candidates specified) are selectively executed to find the best candidate model.

<img src="img/autopilot-steps.png" width="90%" align="left">

**Autopilot Research Paper:   
https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf**

In [86]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/">Hyperparameter Tuning Jobs</a></b>'.format(region)))


In [87]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/">Training Jobs</a></b>'.format(region)))


# The Next Cell Will Show `InProgress` For A Few Minutes.

## _Please be patient._

In [30]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(15)
    print('[OK] Model tuning phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

Completed
MaxCandidatesReached
[OK] Model tuning phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:331041401449:automl-job/automl-dm-11-14-13-48",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 5400,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 900
        }
    },
    "AutoMLJobName": "automl-dm-11-14-13-48",
    "AutoMLJobSecon

# _Please Wait Until ^^ Autopilot ^^ Completes Above_

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

Make sure the status below indicates `Completed`.

In [31]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
print(job_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status not in ('Completed'):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        print(job_status)
        sleep(10)
    print('[OK] Autopilot Job completed.\n')
else:
    print(job_status)


Completed
[OK] Autopilot Job completed.

CPU times: user 3.87 ms, sys: 0 ns, total: 3.87 ms
Wall time: 50 ms


# Viewing All Candidates
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [32]:
candidates_response = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                         SortBy='FinalObjectiveMetricValue')

### Check that candidates exist

In [33]:
print(candidates_response.keys())

dict_keys(['Candidates', 'ResponseMetadata'])


In [34]:
while 'Candidates' not in candidates_response.keys():
    candidates_response = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                         SortBy='FinalObjectiveMetricValue')
    print('[INFO] Autopilot Job is generating the Candidates. Please wait.')
    print(json.dumps(candidates_response, indent=4, sort_keys=True, default=str))
    sleep(10)

candidates = candidates_response['Candidates']
print('[OK] Candidates generated.') 

[OK] Candidates generated.


In [35]:
print(candidates[0].keys())

dict_keys(['CandidateName', 'FinalAutoMLJobObjectiveMetric', 'ObjectiveStatus', 'CandidateSteps', 'CandidateStatus', 'InferenceContainers', 'CreationTime', 'EndTime', 'LastModifiedTime'])


In [36]:
while 'CandidateName' not in candidates[0]:
    candidates_response = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                         SortBy='FinalObjectiveMetricValue')
    candidates = candidates_response['Candidates']
    print('[INFO] Autopilot Job is generating CandidateName. Please wait. ')
    print(json.dumps(candidates, indent=4, sort_keys=True, default=str))
    sleep(10)

print('[OK] CandidateName generated.')     

[OK] CandidateName generated.


In [37]:
while 'FinalAutoMLJobObjectiveMetric' not in candidates[0]:
    candidates_response = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                         SortBy='FinalObjectiveMetricValue')
    candidates = candidates_response['Candidates']
    print('[INFO] Autopilot Job is generating FinalAutoMLJobObjectiveMetric. Please wait. ')
    print(json.dumps(candidates, indent=4, sort_keys=True, default=str))
    sleep(10)

print('[OK] FinalAutoMLJobObjectiveMetric generated.')  

[OK] FinalAutoMLJobObjectiveMetric generated.


In [38]:
print(json.dumps(candidates, indent=4, sort_keys=True, default=str))

[
    {
        "CandidateName": "tuning-job-1-e2f87a329cb24779ad-001-33072cc0",
        "CandidateStatus": "Completed",
        "CandidateSteps": [
            {
                "CandidateStepArn": "arn:aws:sagemaker:us-east-1:331041401449:processing-job/db-1-c746168660554129af899c43592f2144cddc7a48812249d6a5e913fb97",
                "CandidateStepName": "db-1-c746168660554129af899c43592f2144cddc7a48812249d6a5e913fb97",
                "CandidateStepType": "AWS::SageMaker::ProcessingJob"
            },
            {
                "CandidateStepArn": "arn:aws:sagemaker:us-east-1:331041401449:training-job/automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7937c148b5bf0db3c4",
                "CandidateStepName": "automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7937c148b5bf0db3c4",
                "CandidateStepType": "AWS::SageMaker::TrainingJob"
            },
            {
                "CandidateStepArn": "arn:aws:sagemaker:us-east-1:331041401449:transform-job/automl-dm--dpp0-rpb-1-2732a

In [39]:
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

0  tuning-job-1-e2f87a329cb24779ad-001-33072cc0  0.43156999349594116
1  tuning-job-1-e2f87a329cb24779ad-003-39c3a117  0.4232800006866455
2  tuning-job-1-e2f87a329cb24779ad-002-5d4148e8  0.34172001481056213


# Inspect Trials using Experiments API

SageMaker Autopilot automatically creates a new experiment, and pushes information for each trial. 

In [40]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
print(df)

                                  TrialComponentName  \
0  tuning-job-1-e2f87a329cb24779ad-001-33072cc0-a...   
1  tuning-job-1-e2f87a329cb24779ad-002-5d4148e8-a...   
2  tuning-job-1-e2f87a329cb24779ad-003-39c3a117-a...   
3  automl-dm--dpp0-rpb-1-2732a5733bd7403fb716970b...   
4  automl-dm--dpp1-csv-1-e1756ff0505d43f7b1fa0386...   
5  automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7...   
6  automl-dm--dpp1-1-cc308a05319a46749f1052410884...   
7  db-1-c746168660554129af899c43592f2144cddc7a488...   

                                         DisplayName  \
0  tuning-job-1-e2f87a329cb24779ad-001-33072cc0-a...   
1  tuning-job-1-e2f87a329cb24779ad-002-5d4148e8-a...   
2  tuning-job-1-e2f87a329cb24779ad-003-39c3a117-a...   
3  automl-dm--dpp0-rpb-1-2732a5733bd7403fb716970b...   
4  automl-dm--dpp1-csv-1-e1756ff0505d43f7b1fa0386...   
5  automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7...   
6  automl-dm--dpp1-1-cc308a05319a46749f1052410884...   
7  db-1-c746168660554129af899c43592f2144cddc7a4

# Explore the Best Candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). For this notebook, we deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [41]:
best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

In [100]:
print(best_candidate_response.keys())

dict_keys(['AutoMLJobName', 'AutoMLJobArn', 'InputDataConfig', 'OutputDataConfig', 'RoleArn', 'AutoMLJobConfig', 'CreationTime', 'EndTime', 'LastModifiedTime', 'BestCandidate', 'AutoMLJobStatus', 'AutoMLJobSecondaryStatus', 'GenerateCandidateDefinitionsOnly', 'AutoMLJobArtifacts', 'ResolvedAttributes', 'ResponseMetadata'])


In [46]:
while 'BestCandidate' not in best_candidate_response:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print('[INFO] Autopilot Job is generating BestCandidate. Please wait. ')
    print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response['BestCandidate']
print('[OK] BestCandidate generated.')  

[OK] BestCandidate generated.


In [42]:
print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str))

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:331041401449:automl-job/automl-dm-11-14-13-48",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/sagemaker-automl-candidates/pr-1-82ebc5108b8049cf9df8cc241283c077c80e1be33a1c4f47b145b2918d/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 5400,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 900
        }
    },
    "AutoMLJobName": "automl-dm-11-14-13-48",
    "AutoMLJobSecondaryStatus": "MaxCandidatesReached",
    "AutoMLJobStatus": "Comple

In [47]:
print(best_candidate.keys())

dict_keys(['CandidateName', 'FinalAutoMLJobObjectiveMetric', 'ObjectiveStatus', 'CandidateSteps', 'CandidateStatus', 'InferenceContainers', 'CreationTime', 'EndTime', 'LastModifiedTime'])


In [48]:
while 'CandidateName' not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response['BestCandidate']
    print('[INFO] Autopilot Job is generating BestCandidate CandidateName. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print('[OK] BestCandidate CandidateName generated.')  

[OK] BestCandidate CandidateName generated.


In [105]:
while 'FinalAutoMLJobObjectiveMetric' not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response['BestCandidate']
    print('[INFO] Autopilot Job is generating BestCandidate FinalAutoMLJobObjectiveMetric. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print('[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.')  

[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.


In [49]:
best_candidate_identifier = best_candidate['CandidateName']
print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

Candidate name: tuning-job-1-e2f87a329cb24779ad-001-33072cc0
Metric name: validation:accuracy
Metric value: 0.43156999349594116


In [50]:
print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))

{
    "CandidateName": "tuning-job-1-e2f87a329cb24779ad-001-33072cc0",
    "CandidateStatus": "Completed",
    "CandidateSteps": [
        {
            "CandidateStepArn": "arn:aws:sagemaker:us-east-1:331041401449:processing-job/db-1-c746168660554129af899c43592f2144cddc7a48812249d6a5e913fb97",
            "CandidateStepName": "db-1-c746168660554129af899c43592f2144cddc7a48812249d6a5e913fb97",
            "CandidateStepType": "AWS::SageMaker::ProcessingJob"
        },
        {
            "CandidateStepArn": "arn:aws:sagemaker:us-east-1:331041401449:training-job/automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7937c148b5bf0db3c4",
            "CandidateStepName": "automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7937c148b5bf0db3c4",
            "CandidateStepType": "AWS::SageMaker::TrainingJob"
        },
        {
            "CandidateStepArn": "arn:aws:sagemaker:us-east-1:331041401449:transform-job/automl-dm--dpp0-rpb-1-2732a5733bd7403fb716970b298f65dd25dac1e7a",
            "CandidateStepN

# View Individual Autopilot Jobs

In [62]:
while 'CandidateSteps' not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response['BestCandidate']
    print('[INFO] Autopilot Job is generating BestCandidate CandidateSteps. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response['BestCandidate']
print('[OK] BestCandidate CandidateSteps generated.')

[OK] BestCandidate CandidateSteps generated.


In [63]:
while 'CandidateStepType' not in best_candidate['CandidateSteps'][0]:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response['BestCandidate']
    print('[INFO] Autopilot Job is generating BestCandidate CandidateSteps CandidateStepType. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response['BestCandidate']
print('[OK] BestCandidate CandidateSteps CandidateStepType generated.')

[OK] BestCandidate CandidateSteps CandidateStepType generated.


In [73]:
while 'CandidateStepName' not in best_candidate['CandidateSteps'][0]:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response['BestCandidate']
    print('[INFO] Autopilot Job is generating BestCandidate CandidateSteps CandidateStepName. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response['BestCandidate']
print('[OK] BestCandidate CandidateSteps CandidateStepName generated.')

[OK] BestCandidate CandidateSteps CandidateStepName generated.


In [51]:
steps = []
for step in best_candidate['CandidateSteps']:
    print('Candidate Step Type: {}'.format(step['CandidateStepType']))
    print('Candidate Step Name: {}'.format(step['CandidateStepName']))
    steps.append(step['CandidateStepName'])

Candidate Step Type: AWS::SageMaker::ProcessingJob
Candidate Step Name: db-1-c746168660554129af899c43592f2144cddc7a48812249d6a5e913fb97
Candidate Step Type: AWS::SageMaker::TrainingJob
Candidate Step Name: automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7937c148b5bf0db3c4
Candidate Step Type: AWS::SageMaker::TransformJob
Candidate Step Name: automl-dm--dpp0-rpb-1-2732a5733bd7403fb716970b298f65dd25dac1e7a
Candidate Step Type: AWS::SageMaker::TrainingJob
Candidate Step Name: tuning-job-1-e2f87a329cb24779ad-001-33072cc0


In [66]:
from IPython.core.display import display, HTML

display(HTML('<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, steps[0])))

In [110]:
from IPython.core.display import display, HTML

display(HTML('<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a></b>'.format(region, steps[1])))

In [109]:
from IPython.core.display import display, HTML

display(HTML('<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/{}">Transform Job</a></b>'.format(region, steps[2])))

In [72]:
from IPython.core.display import display, HTML

display(HTML('<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job (Tuning)</a></b>'.format(region, steps[3])))

# See the containers and models composing the Inference Pipeline

In [53]:
while 'InferenceContainers' not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response['BestCandidate']
    print('[INFO] Autopilot Job is generating BestCandidate InferenceContainers. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print('[OK] BestCandidate InferenceContainers generated.') 

[OK] BestCandidate InferenceContainers generated.


In [54]:
best_candidate_containers = best_candidate['InferenceContainers']

In [55]:
for container in best_candidate_containers:
        print(container['Image'])
        print(container['ModelDataUrl'])
        print('======================')

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.2-1-cpu-py3
s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/data-processor-models/automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7937c148b5bf0db3c4/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3
s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/tuning/automl-dm--dpp0-xgb/tuning-job-1-e2f87a329cb24779ad-001-33072cc0/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.2-1-cpu-py3
s3://ada-dev/xiaotian/ml/autopilot/model/automl-dm-11-14-13-48/data-processor-models/automl-dm--dpp0-1-da688f7f84d54dcf8469f92d64f7937c148b5bf0db3c4/output/model.tar.gz


# Update Containers To Show Predicted Label and Confidence Score

In [114]:
for container in best_candidate_containers:
        print(container['Environment'])
        print('======================')

{'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
{'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities'}
{'AUTOML_TRANSFORM_MODE': 'inverse-label-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_INPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,labels,probabilities', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}


In [79]:
best_candidate_containers[1]['Environment'].update({'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability'})
best_candidate_containers[2]['Environment'].update({'SAGEMAKER_INFERENCE_INPUT': 'predicted_label, probability'})
best_candidate_containers[2]['Environment'].update({'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability'})

In [80]:
for container in best_candidate_containers:
        print(container['Environment'])
        print('======================')

{'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
{'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities'}
{'AUTOML_TRANSFORM_MODE': 'inverse-label-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_INPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,labels,probabilities', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}


# Autopilot Chooses XGBoost as Best Candidate!

Note that Autopilot chose different hyper-parameters and feature transformations than we used in our own XGBoost model.

# Deploy the Model as a REST Endpoint
Batch transformations are also supported, but for now, we will use a REST Endpoint.

In [77]:
print(best_candidate['InferenceContainers'])

[{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.2-1-cpu-py3', 'ModelDataUrl': 's3://ada-dev/xiaotian/ml/autopilot/automl-dm-11-03-26-46/data-processor-models/automl-dm--dpp2-1-ef2b1364d78443cc95fd8071bd99f0d7ec5d91b36bac4/output/model.tar.gz', 'Environment': {'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}}, {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3', 'ModelDataUrl': 's3://ada-dev/xiaotian/ml/autopilot/automl-dm-11-03-26-46/tuning/automl-dm--dpp2-xgb/tuning-job-1-4030f05282354943b7-001-a516fb96/output/model.tar.gz', 'Environment': {'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE

In [78]:
%store -r autopilot_model_name

no stored variable or alias autopilot_model_name


In [79]:
try:
    autopilot_model_name
except NameError:    
    timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
    autopilot_model_name = 'automl-dm-model-' + timestamp_suffix
    print('[OK] Created Autopilot Model Name: ' + autopilot_model_name)

[OK] Created Autopilot Model Name: automl-dm-model-11-07-13-51


In [ ]:
%store autopilot_model_name

In [ ]:
%store -r autopilot_model_arn

In [ ]:
try:
    autopilot_model_arn
except NameError:
    create_model_response = sm.create_model(Containers=best_candidate['InferenceContainers'],
                                ModelName=autopilot_model_name,
                                ExecutionRoleArn=role)
    autopilot_model_arn = create_model_response['ModelArn']
    print('[OK] Created Autopilot Model: {}'.format(autopilot_model_arn))

In [ ]:
%store autopilot_model_arn

# Define EndpointConfig Name

In [ ]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

print(epc_name)

# Define REST Endpoint Name for Autopilot Model

In [ ]:
%store -r autopilot_endpoint_name

In [ ]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

try:
    autopilot_endpoint_name
except NameError:    
    autopilot_endpoint_name = 'automl-dm-ep-' + timestamp_suffix
    print('[OK] Created Autopilot Endpoint Name {}: '.format(autopilot_endpoint_name))    

In [ ]:
variant_name = 'automl-dm-variant-' + timestamp_suffix
print('[OK] Created Endpoint Variant Name {}: '.format(variant_name))

In [ ]:
%store autopilot_endpoint_name

In [ ]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m5.large',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': autopilot_model_name,
                                                           'VariantName': variant_name}])

In [ ]:
%store -r autopilot_endpoint_arn

In [ ]:
try:
    autopilot_endpoint_arn
except NameError:    
    create_endpoint_response = sm.create_endpoint(EndpointName=autopilot_endpoint_name,
                                                  EndpointConfigName=epc_name) 
    autopilot_endpoint_arn = create_endpoint_response['EndpointArn']

In [ ]:
%store autopilot_endpoint_arn

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, autopilot_endpoint_name)))

# Store Variables for the Next Notebooks

In [ ]:
%store

# Summary
We used Autopilot to automatically find the best model, hyper-parameters, and feature-engineering scripts for our dataset.  

Autopilot uses a transparent approach to generate re-usable exploration Jupyter Notebooks and transformation Python scripts to continue to train and deploy our model on new data - well after this initial interaction with the Autopilot service.

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();